In [1]:
import os
import sys
sys.path.append('../')
os.chdir('../')

In [2]:
import duckdb 
import pandas as pd
import duck_db_helper

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
conn = duckdb.connect('../data/trusted_zone/trusted_zone.db')
tables = duck_db_helper.get_tables(conn)
df_list = []
df_list_names = []
for table_name in tables: 
    df = conn.sql(f"SELECT * FROM \"{table_name}\";").df()
    df_list.append(df)
    df_list_names.append(table_name)
conn.close()

In [4]:
df_list_names

['Metoffice',
 'cleaned_merged_seasons',
 'football-data',
 'team_stadium_location',
 'weather_station_locations']

In [5]:
def get_index(table):
    for i,name in enumerate(df_list_names):
        if table in name: 
            return i
    return -1

In [6]:
df_list[get_index('Metoffice')].head()

,Date,Station_no,Station_name,PRESS,WDIR,WSPD,CLOUD,TEMP,TDEW
0,2023-05-01,1,Lerwick,1012.1,ENE,18.000000,8.0,4.8,4.4
1,2023-05-01,2,Wick,1010.7,W,6.000000,8.0,10.2,9.9
2,2023-05-01,3,Stornoway,1013.2,W,10.000000,8.0,9.6,9.0
3,2023-05-01,4,Loch Glascarnoch,1013.0,W,14.000000,8.0,9.5,7.9
4,2023-05-01,5,Aviemore,1013.2,nan,8.333333,8.0,9.0,7.4


In [7]:
df_list[get_index('cleaned_merged_seasons')].head()

,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,opp_team_name,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,2016-17,Aaron Cresswell,DEF,None,0,0,0,0,0.0,454,10,0,0,0.0,0.0,2016-08-15 21:00:00+02:00,0,4,Chelsea,0,0,0,0,1,0,14023,1,2,0,0,0,0,0,55,0,0,1
1,2016-17,Aaron Lennon,MID,None,0,0,6,0,0.3,142,3,0,0,0.9,8.2,2016-08-13 16:00:00+02:00,15,17,Spurs,0,0,0,0,1,0,13918,1,1,0,1,0,0,0,60,1,0,1
2,2016-17,Aaron Ramsey,MID,None,0,0,5,0,4.9,16,8,3,0,3.0,2.2,2016-08-14 17:00:00+02:00,60,9,Liverpool,0,0,0,0,1,0,163170,4,3,23,2,0,0,0,80,1,0,1
3,2016-17,Abdoulaye Doucouré,MID,None,0,0,0,0,0.0,482,7,0,0,0.0,0.0,2016-08-13 16:00:00+02:00,0,13,Southampton,0,0,0,0,1,0,1051,1,1,0,0,0,0,0,50,0,0,1
4,2016-17,Adam Forshaw,MID,None,0,0,3,0,1.3,286,6,1,0,0.3,2.0,2016-08-13 16:00:00+02:00,69,14,Stoke,0,0,0,0,1,0,2723,1,1,0,1,0,0,0,45,1,1,1


In [8]:
df_list[get_index('football-data')].head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,2022-05-08,2023-10-21 20:00:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,10,10,2,2,16,11,3,5,1,2,0,0,4.20,3.6,1.85,4.33,3.50,1.87,4.30,3.55,1.85,4.50,3.65,1.89,4.40,3.5,1.83,4.60,3.5,1.87,4.60,3.78,1.95,4.39,3.59,1.88,2.10,1.72,2.14,1.78,2.19,1.91,2.09,1.76,0.50,2.04,1.89,2.03,1.89,2.06,1.91,2.01,1.87,4.50,3.60,1.80,4.50,3.5,1.83,4.40,3.55,1.85,4.58,3.63,1.88,4.80,3.4,1.78,4.75,3.5,1.85,5.01,3.70,1.91,4.56,3.57,1.85,2.10,1.72,2.14,1.78,2.19,1.91,2.08,1.76,0.50,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85
1,E0,2022-06-08,2023-10-21 12:30:00,Fulham,Liverpool,2,2,D,1,0,H,A Madley,9,11,3,4,7,9,4,4,2,0,0,0,11.00,6.0,1.25,10.00,5.75,1.28,12.00,5.75,1.27,11.20,6.22,1.28,12.00,5.5,1.27,13.00,6.0,1.25,13.00,6.40,1.31,10.99,6.05,1.28,1.50,2.62,1.50,2.70,1.54,2.76,1.48,2.63,1.75,1.90,2.03,1.91,2.00,1.92,2.04,1.89,1.99,11.00,5.75,1.28,9.25,6.0,1.29,11.00,5.50,1.30,10.50,6.50,1.29,11.00,5.5,1.27,11.50,6.0,1.29,11.95,6.93,1.30,10.33,6.20,1.28,1.50,2.62,1.49,2.77,1.51,3.00,1.47,2.73,1.75,1.90,2.03,1.91,2.02,2.01,2.06,1.89,1.99
2,E0,2022-06-08,2023-10-21 15:00:00,Bournemouth,Aston Villa,2,0,H,1,0,H,P Bankes,7,15,3,2,18,16,5,5,3,3,0,0,3.75,3.5,2.00,3.75,3.40,2.00,3.65,3.45,2.05,3.93,3.58,2.04,3.75,3.3,2.05,3.75,3.3,2.00,4.00,3.66,2.10,3.80,3.50,2.04,2.00,1.80,2.10,1.81,2.10,1.87,2.03,1.80,0.50,1.87,2.06,1.88,2.04,1.88,2.07,1.85,2.04,4.00,3.50,1.95,3.90,3.4,1.95,3.85,3.45,2.00,4.09,3.59,2.00,4.00,3.4,1.95,4.10,3.4,2.00,4.25,3.63,2.06,3.99,3.49,2.00,2.10,1.72,2.13,1.79,2.24,1.81,2.10,1.76,0.50,1.93,2.00,1.93,2.00,1.94,2.04,1.88,2.00
3,E0,2022-06-08,2023-10-21 15:00:00,Leeds,Wolves,2,1,H,1,1,D,R Jones,12,15,4,6,13,9,6,4,2,0,0,0,2.25,3.4,3.20,2.30,3.30,2.95,2.30,3.30,3.15,2.39,3.33,3.30,2.25,3.3,3.20,2.30,3.2,3.10,2.42,3.54,3.30,2.34,3.34,3.18,2.05,1.85,2.09,1.83,2.11,1.87,2.03,1.81,-0.25,2.05,1.88,2.04,1.89,2.06,1.90,2.01,1.87,2.37,3.30,3.00,2.40,3.3,2.75,2.45,3.30,2.95,2.45,3.44,3.09,2.40,3.3,2.90,2.40,3.4,3.00,2.50,3.55,3.18,2.43,3.36,3.02,1.95,1.95,1.96,1.94,2.09,1.96,1.96,1.87,-0.25,2.08,1.85,2.10,1.84,2.14,1.87,2.08,1.81
4,E0,2022-06-08,2023-10-21 15:00:00,Newcastle,Nott'm Forest,2,0,H,0,0,D,S Hooper,23,5,10,0,9,14,11,1,0,3,0,0,1.66,3.8,5.25,1.65,3.80,5.50,1.65,3.80,5.50,1.71,3.74,5.83,1.67,3.7,5.25,1.62,3.7,5.50,1.72,3.96,6.00,1.67,3.80,5.57,2.05,1.85,2.10,1.81,2.10,1.86,2.03,1.81,-0.75,1.87,2.06,1.92,2.01,1.92,2.08,1.86,2.02,1.53,4.00,6.00,1.58,3.9,6.00,1.63,3.80,6.00,1.57,4.22,6.60,1.53,3.9,6.50,1.57,3.9,7.00,1.67,4.30,7.00,1.59,4.07,6.15,1.94,1.96,1.97,1.93,2.06,1.97,1.94,1.89,-1.00,1.97,1.96,1.99,1.93,2.19,1.97,2.03,1.86


In [10]:
df_list[get_index('team_stadium_location')].head()

,team_name,NAME,LAT,LON
0,Arsenal,Emirates Stadium,51.558920,-0.109781
1,Aston Villa,Villa Park,52.509203,-1.884718
2,Bournemouth,Vitality Stadium,50.734974,-1.839025
3,Brentford,Gtech Gommunity Stadium,51.490959,-0.288670
4,Brighton,American Express Community Stadium,50.861748,-0.083706


In [11]:
df_list[get_index('weather_station_locations')].head()

,SITE,LAT,LON
0,Lerwick,60.190187,-1.088959
1,Wick,58.463990,-3.081510
2,Stornoway,58.213929,-6.387110
3,Loch Glascarnoch,57.769856,-4.852771
4,Aviemore,57.258025,-3.868066
